In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [ ]:
# Mesh related functions
def create_grid(xi, xf, Ns, dim=2):
    """
        Create a uniform mesh in either 1D or 2D.
        For 2D, the 1D mesh is simply extended to 2D.
    """
    x, dx = np.linspace(xi, xf, Ns, retstep=True)
    p = 2*np.pi*np.fft.fftfreq(Ns, d=dx)
    dp = p[1]-p[0]
    if dim == 2:
        x = np.meshgrid(x, x)
        p = np.meshgrid(p, p)
    return x, dx, p, dp



def integrate(f, dx):
    """
        Integrate a function (1D or 2D) using trapeziodal integration.
    """
    if len(f.shape) == 2:
        return np.trapz(np.trapz(f, dx=dx), dx=dx)
    else:
        return np.trapz(f, dx=dx)

In [ ]:
# Fixed quantum related functions
def H(K, V, g, Omega, x, p, psi):
    """
        Return Hamilotnian applied to wavefunction.
        Support for 2D and 1D (if Omega is zero).
    """
    Hpsi = np.fft.ifftn(K*np.fft.fftn(psi))  # Kinetic
    Hpsi += (V+g*np.abs(psi)**2)*psi         # Trap
    if Omega != 0.0:
        Hpsi += np.fft.ifftn(-Omega*x[1]*p[0]*np.fft.fftn(psi, axes=(1,)), axes=(1,))
        Hpsi += np.fft.ifftn(Omega*x[0]*p[1]*np.fft.fftn(psi, axes=(0,)), axes=(0,))
    return  Hpsi


def energy(K, V, g, Omega, x, p, psi, dx):
    """
        Return energy of psi.
        See H(...) for more info.
    """
    return integrate(np.real(np.conjugate(psi)*H(K, V, g, Omega, x, p, psi)), dx)


def density_flux(psi):
    """
        Return the quantum flux.
    """
    gpsi = np.gradient(psi)
    gpsic = np.gradient(np.conjugate(psi))
    return np.real(0.5j*(psi*gpsic[0]-np.conjugate(psi)*gpsi[0])),\
           np.real(0.5j*(psi*gpsic[1]-np.conjugate(psi)*gpsi[1]))


def solve(K, V, g, Omega, x, dx, p, dp, psi0, dt, nsteps):
    """
        Perform imaginary time-evolution.
    """
    t = np.zeros(nsteps)
    e = np.zeros(nsteps)
    Up = np.exp(-0.5*dt*K)
    if Omega != 0.0:
        Uxpy = np.exp(0.5*dt*Omega*x[1]*p[0])
        Uypx = np.exp(-dt*Omega*x[0]*p[1])
    psi = np.copy(psi0)
    e[0] = energy(K, V, g, Omega, x, p, psi, dx)
    for i in range(1, nsteps):
        # Trap+Non-Linearity (half)
        Ux = np.exp(-0.5*dt*(V + g*np.abs(psi)**2))
        psi *= Ux
        # Kinetic Energy (half)
        psi = np.fft.fftn(psi)
        psi *= Up
        if Omega != 0.0:
            psi = np.fft.ifftn(psi, axes=(0,))
            # Rotation (xpy) (half)
            psi *= Uxpy
            psi = np.fft.ifftn(psi, axes=(1,))
            # Rotation (ypx) (full)
            psi = np.fft.fftn(psi, axes=(0,))
            psi *= Uypx
            psi = np.fft.ifftn(psi, axes=(0,))
            # Rotation (xpy) (half)
            psi = np.fft.fftn(psi, axes=(1,))
            psi *= Uxpy
            # Kinetic Energy (half)
            psi = np.fft.fftn(psi, axes=(0,))
        psi *= Up
        psi = np.fft.ifftn(psi)
        # Trap+Non-Linearity (half)
        psi *= Ux
        # Normalization
        psi /= integrate(np.abs(psi)**2, dx)**0.5
        e[i] = energy(K, V, g, Omega, x, p, psi, dx)
        t[i] = t[i-1]+abs(dt)

    return psi, e, t

In [ ]:
# Fixed parameters
Ns = 2**7  # Grid size
N = 1e3    # Number of particles

In [ ]:
# Set trap
g = 1.0*N
xi, xf = (-8, 8)
x, dx, p, dp = create_grid(xi, xf, Ns)

def Vfunc(x, a):
    if type(x) is list:
        x2 = np.sum([xd**2 for xd in x], axis=0)
    else:
        x2 = x**2
    # return -a*x2+x2**2
    # return (x2**0.5-a)**2
    return 0.5*x2+a*np.exp(-0.5*x2)-a

# Kinetic and trap operators
K = 0.5*np.sum([pd**2 for pd in p], axis=0)

In [ ]:
def initial_guess(V, initial_TF, l):
    if not initial_TF:
        psi0 = np.exp(-0.5*(x[0]**2+x[1]**2), dtype=complex)  # initial guess
        psi0 *= np.exp(1.0j*l*np.arctan2(x[1], x[0]))
    else:
        def psiTF(mu):
            res = (mu*np.ones(V.shape, dtype=complex)-V) / g
            res[res < 0] = 0
            return np.sqrt(res)

        res = opt.minimize(lambda mu: abs(integrate(psiTF(mu)**2, dx)-1)**2, 1)
        psi0 = psiTF(res.x)
        print(f"ENE_TF = {res.x[0]}")

    psi0 *= np.exp(-1.0j*l*np.arctan2(x[1], x[0])+0.5j)*(np.exp(-10.0*(x[0]**2+x[1]**2))-1)

    psi0 /= integrate(np.abs(psi0)**2, dx)**0.5
    plt.imshow(np.abs(psi0)**2)
    return psi0

In [ ]:
def run(a, Omega, psi0, dt, nsteps):
    V = Vfunc(x, a)
    return solve(K, V, g, Omega, x, dx, p, dp, psi0, dt, nsteps)


In [ ]:
psi = initial_guess(Vfunc(x, 0), True, 3)
dt = 0.0005
Omega = 0.4

In [ ]:
a = 0.0
psi, e, t = run(a, Omega, psi, dt, 100000)

print(f"ENERGY = {e[-1]}")
print(f"ERROR (WFC)  = {(integrate(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi)-e[-1]*psi)**2, dx)/e[-1]**2)**0.5}")
print(f"ERROR (Norm) = {integrate(np.abs(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi))**2 - np.abs(e[-1]*psi)**2), dx)**0.5/e[-1]}")

fig, axes = plt.subplots(1, 4, figsize=(11, 3))
axes[0].plot(t, e)
axes[1].imshow(np.abs(psi)**2)
axes[1].axis('off')
axes[2].imshow(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi))**2)
axes[2].axis('off')
axes[3].imshow(np.angle(psi), cmap='twilight_shifted', interpolation='none')
axes[3].axis('off')
plt.tight_layout()
plt.show()

flux = density_flux(psi)
fig, axes = plt.subplots(2, 1, figsize=(3, 6))
axes[0].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
axes[1].streamplot(*x, -flux[1], -flux[0], linewidth=2*np.abs(psi)**2/np.max(np.abs(psi)**2), color='k')
axes[1].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
for axis in axes:
    axis.axis('off')
    axis.axis('square')
plt.tight_layout()
plt.show()

flux = density_flux(psi)
fig, axes = plt.subplots(2, 1, figsize=(3, 6))
axes[0].streamplot(*x, -flux[1], -flux[0], linewidth=2*np.abs(psi)**2/np.max(np.abs(psi)**2), color='k')
axes[0].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot', aspect="auto")
axes[1].imshow(np.angle(psi), extent=[xi, xf, xi, xf], origin='lower', cmap='twilight_shifted', interpolation='none')
for axis in axes:
    axis.axis('off')
    axis.axis('square')
plt.tight_layout()
plt.show()

In [ ]:
for i in range(0, 150):
    psi, e, t = run(i/10.0, Omega, psi, dt, 1000)

In [ ]:
a = 15.0
psi, e, t = run(a, Omega, psi, dt, 100000)

print(f"ENERGY = {e[-1]}")
print(f"ERROR (WFC)  = {(integrate(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi)-e[-1]*psi)**2, dx)/e[-1]**2)**0.5}")
print(f"ERROR (Norm) = {integrate(np.abs(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi))**2 - np.abs(e[-1]*psi)**2), dx)**0.5/e[-1]}")

fig, axes = plt.subplots(1, 4, figsize=(11, 3))
axes[0].plot(t, e)
axes[1].imshow(np.abs(psi)**2)
axes[1].axis('off')
axes[2].imshow(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi))**2)
axes[2].axis('off')
axes[3].imshow(np.angle(psi), cmap='twilight_shifted', interpolation='none')
axes[3].axis('off')
plt.tight_layout()
plt.show()

flux = density_flux(psi)
fig, axes = plt.subplots(2, 1, figsize=(3, 6))
axes[0].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
axes[1].streamplot(*x, -flux[1], -flux[0], linewidth=2*np.abs(psi)**2/np.max(np.abs(psi)**2), color='k')
axes[1].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
for axis in axes:
    axis.axis('off')
    axis.axis('square')
plt.tight_layout()
plt.show()

flux = density_flux(psi)
fig, axes = plt.subplots(2, 1, figsize=(3, 6))
axes[0].streamplot(*x, -flux[1], -flux[0], linewidth=2*np.abs(psi)**2/np.max(np.abs(psi)**2), color='k')
axes[0].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot', aspect="auto")
axes[1].imshow(np.angle(psi), extent=[xi, xf, xi, xf], origin='lower', cmap='twilight_shifted', interpolation='none')
for axis in axes:
    axis.axis('off')
    axis.axis('square')
plt.tight_layout()
plt.show()

In [ ]:
for i in range(150, 300):
    psi, e, t = run(i/1.0, Omega, psi, dt, 1000)

In [ ]:
a = 30.0
psi, e, t = run(a, Omega, psi, dt, 100000)

print(f"ENERGY = {e[-1]}")
print(f"ERROR (WFC)  = {(integrate(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi)-e[-1]*psi)**2, dx)/e[-1]**2)**0.5}")
print(f"ERROR (Norm) = {integrate(np.abs(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi))**2 - np.abs(e[-1]*psi)**2), dx)**0.5/e[-1]}")

fig, axes = plt.subplots(1, 4, figsize=(11, 3))
axes[0].plot(t, e)
axes[1].imshow(np.abs(psi)**2)
axes[1].axis('off')
axes[2].imshow(np.abs(H(K, Vfunc(x, a), g, Omega, x, p, psi))**2)
axes[2].axis('off')
axes[3].imshow(np.angle(psi), cmap='twilight_shifted', interpolation='none')
axes[3].axis('off')
plt.tight_layout()
plt.show()

flux = density_flux(psi)
fig, axes = plt.subplots(2, 1, figsize=(3, 6))
axes[0].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
axes[1].streamplot(*x, -flux[1], -flux[0], linewidth=2*np.abs(psi)**2/np.max(np.abs(psi)**2), color='k')
axes[1].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
for axis in axes:
    axis.axis('off')
    axis.axis('square')
plt.tight_layout()
plt.show()

flux = density_flux(psi)
fig, axes = plt.subplots(2, 1, figsize=(3, 6))
axes[0].streamplot(*x, -flux[1], -flux[0], linewidth=2*np.abs(psi)**2/np.max(np.abs(psi)**2), color='k')
axes[0].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot', aspect="auto")
axes[1].imshow(np.angle(psi), extent=[xi, xf, xi, xf], origin='lower', cmap='twilight_shifted', interpolation='none')
for axis in axes:
    axis.axis('off')
    axis.axis('square')
plt.tight_layout()
plt.show()

In [ ]:
def periodic_gradient(f, dx):
    res = [np.zeros(f.shape), np.zeros(f.shape)]
    fip = np.roll(f, 1, axis=0)
    fim = np.roll(f, -1, axis=0)
    fjp = np.roll(f, 1, axis=1)
    fjm = np.roll(f, -1, axis=1)
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            v1 = (fip[i, j]-fim[i, j]) / (2*dx)
            v2 = (fip[i, j]-fim[i, j]+2*np.pi) / (2*dx)
            v3 = (fip[i, j]-fim[i, j]) / (2*dx)
            res[0][i, j] = v1 if abs(v1) < abs(v2) else v2 if abs(v2) < abs(v3) else v3
            v1 = (fjp[i, j]-fjm[i, j]) / (2*dx)
            v2 = (fjp[i, j]-fjm[i, j]+2*np.pi) / (2*dx)
            v3 = (fjp[i, j]-fjm[i, j]) / (2*dx)
            res[1][i, j] = v1 if abs(v1) < abs(v2) else v2 if abs(v2) < abs(v3) else v3
    return res

flux = density_flux(psi)
fig, axes = plt.subplots(3, 1, figsize=(4, 12))
axes[0].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
axes[1].streamplot(*x, -flux[1], -flux[0], linewidth=2*np.abs(psi)**2/np.max(np.abs(psi)**2), color='k')
axes[1].imshow(np.abs(psi)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
axes[2].imshow(np.angle(psi), extent=[xi, xf, xi, xf], origin='lower', cmap='twilight_shifted', interpolation='none')
for axis in axes:
    axis.axis('off')
    axis.axis('square')
plt.tight_layout()
plt.show()

In [ ]:
def get_TF(V):
    def psiTF(mu):
        res = (mu*np.ones(V.shape, dtype=complex)-V) / g
        res[res < 0] = 0
        return np.sqrt(res)

    res = opt.minimize(lambda mu: abs(integrate(psiTF(mu)**2, dx)-1)**2, 1)
    return psiTF(res.x), res.x

a_all = [0, 15, 30]
for a in a_all:
    VT = Vfunc(x, a)
    xT = np.linspace(xi, xf, Ns)
    V1T = Vfunc(xT, a)
    psiT, mu = get_TF(VT)
    fig, axes = plt.subplots(2, 1, figsize=(3, 6))
    axes[0].plot(xT, V1T, c='k')
    axes[0].fill_between(xT, V1T, mu, where=V1T < mu, color='gray')
    axes[1].imshow(np.abs(psiT)**2, extent=[xi, xf, xi, xf], origin='lower', cmap='afmhot')
    axes[0].axis('off')
    axes[1].axis('square')
    axes[1].axis('off')
